# Часть 1

Поставить python и все библиотеки

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score

import smallestenclosingcircle
import numpy as np

In [3]:
iris = load_iris()

X = iris.data[:, :2]
y = iris.target

print(round(np.mean(cross_val_score(RandomForestClassifier(random_state=42), X, y, cv=3)), 4))

0.7132


#### Вопрос 1. Что выведет этот код?

# Часть 2


In [4]:
iris = load_iris()

X = iris.data[:, :2]
y = iris.target

In [12]:
smallestenclosingcircle.make_circle(points=X)

(6.155223880597015, 3.1514925373134326, 1.861398838508048)

#### Вопрос 2. Каков минимальный радиус круга, содержащего все точки из $X$? Ответ округлите до 4 знака после запятой.

https://ru.wikipedia.org/wiki/Пауль_(осьминог)

Этот осьминог предсказывал результаты футбольных матчей на  ЧЕ-2008 и ЧМ-2010. Нужно проанализировать его предсказания, считая, что футбольный матч обязательно заканчивается победой одной из команд, а осьминог возможно выбирал равновероятно наугад.

#### Вопрос 3. Какова вероятность получить именно те предсказания, что дал осьминог, выбирая наугад? Ответ округлите до 5 знака после запятой.

In [30]:
1/(2**14)

6.103515625e-05

#### Вопрос 4. Какова вероятность ошибиться не меньшее число раз, чем осьминог Пауль, выбирая наугад? Ответ округлите до 5 знака после запятой.

In [31]:
(1 + 14 + 14*13//2)/2**14

0.0064697265625

# Часть 3


#### Вопрос 5. У вас есть набор зависимости $y$ от одной переменной $x$, причём вы знаете, что эта зависимость линейная ($y=ax + b$). Вам нужно восстановить эту зависимость, решив оптимизационную задачу $\min_{a, b} \sum_{i=1}^n (ax_i + b - y_i)^2$.

#### Проблема в том, что выборка очень большая, поэтому её нельзя полностью загрузить в память. Ответом на вопрос является строка в питоновском формате '{}_{}'.format(round(a, 5), round(b, 5)), например, '0.00001_0.00002'.

In [158]:
def generator():
    rand_gen = np.random.RandomState(42)
    for i in range(100000):
        arr_of_x = 10 * rand_gen.uniform(size=10000)
        arr_of_noise = 40 * rand_gen.normal(size=10000)
        for x, noise in zip(arr_of_x, arr_of_noise):
            yield x, 3.14 * x + 8.3 + noise

In [159]:
def solver(seq):
    # Напишите здесь свой код
    init_points = [next(seq) for i in range(4)]
    X_init = np.array([np.array([x[0], 1]) for x in init_points])
    y_init = np.array([np.array([x[1]]) for x in init_points])
    
    MTM = np.dot(X_init.T, X_init)
    MTY = np.dot(X_init.T, y_init).flatten()
    print(MTM)
    for num, point in enumerate(seq):
        add_x = np.array([[point[0], 1]])
        add_y = point[1]
        MTM = MTM * (num)/(num+1) + np.dot(add_x.T, add_x)/(num+1)
        MTY = MTY * (num)/(num+1) + (add_x*add_y).reshape((2,))/(num+1)
    ret = np.dot(np.linalg.inv(MTM), MTY)
    return ret[0], ret[1]
        

In [161]:
%%time
a, b = solver(generator())
print('{}_{}'.format(round(a, 5), round(b, 5)))

[[ 193.83451046   26.55906851]
 [  26.55906851    4.        ]]
3.13957_8.30331
CPU times: user 5h 4min 17s, sys: 5min 40s, total: 5h 9min 58s
Wall time: 5h 10min 1s


#### Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

#### В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через "," затем идёт ";" после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, "1,2,3,4;" или "1,2,3,4;5,6". Гарантируется, что среди id купленных товаров все различные.

#### Вопросы 6-7. Будем считать, что цена товара равна его id % 100 + 1. Найдите разницу денежного recall для сортировки по вероятности покупки (методы уже реализованы) без умножения на цену и с умножением на цену для train и для test. Ответ округлите до 5 знака после запятой.

#### У вас есть код для подсчёта обычного recall, вам нужно переписать его, взвесив объекты их ценой. Знаменатель будет равен сумме цен купленных товаров.



In [1]:
from collections import Counter
from collections import OrderedDict
import numpy as np
import pandas
import heapq

In [2]:
visit_popularity = Counter()
purchase_popularity = Counter()

with open('train.txt', 'r') as f:
    for line in f.readlines():
        visits, purchases = line.strip().split(';')
        for visit in visits.split(','):
            visit_popularity[visit] += 1
        if purchases != '':
            for purchase in purchases.split(','):
                purchase_popularity[purchase] += 1

                
def get_item_price(item_id):
    return int(item_id) % 100 + 1

In [3]:
def recommend_by_purchase(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: purchase_popularity.get(x, 0))

def recommend_by_purchase_price_aware(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: get_item_price(x) * purchase_popularity.get(x, 0))

In [4]:
# Допишите код в функции, чтобы считался recall, взвешенный ценами

def calculate_metrics(recommend, max_count, sessions_file):
    with open(sessions_file, 'r') as f:
        avg_recall = np.zeros(max_count)
        sessions_count = 0
        for line in f.readlines():
            visits, purchases = line.strip().split(';')
            if purchases != '':
                visits = visits.split(',')
                purchases = set(purchases.split(','))
                prices_sum_pur = np.array([get_item_price(i) for i in purchases]).sum()
                
                rec = recommend(visits, max_count)
                rec_hits = np.array([(x in purchases) for x in rec])
                hits = np.zeros(max_count)
                prices = np.zeros(max_count)
                hits[:len(rec_hits)] = rec_hits
                
                prices[:len(rec_hits)] = np.array([get_item_price(i) for i in rec])
                hits *= prices
                sessions_count += 1
                avg_recall +=  np.cumsum(hits).astype(np.float32) / (prices_sum_pur)
    return pandas.DataFrame({
            'k': np.arange(max_count) + 1,
            'avg_recall@k': avg_recall / sessions_count,
    }).set_index('k')

In [5]:
print(round((
    calculate_metrics(recommend_by_purchase_price_aware, 3, 'train.txt') 
    - calculate_metrics(recommend_by_purchase, 3, 'train.txt')
)['avg_recall@k'][3], 5))

0.00973


In [6]:
print(round((
    calculate_metrics(recommend_by_purchase_price_aware, 3, 'test.txt') 
    - calculate_metrics(recommend_by_purchase, 3, 'test.txt')
)['avg_recall@k'][3], 5))

-0.00273


In [167]:
calculate_metrics(recommend_by_purchase_price_aware, 3, 'train.txt')

,avg_recall@k
k,
1,0.645221
2,0.754791
3,0.794493


In [168]:
calculate_metrics(recommend_by_purchase, 3, 'train.txt')

,avg_recall@k
k,
1,0.646438
2,0.756991
3,0.790948
